In [1]:
import json

In [2]:
with open('./train_20170724.json', 'r') as f:
    data = json.load(f)

In [3]:
def select_bot(users):
    user1, user2 = users
    if user1['userType'] == 'Human':
        return user2
    else:
        return user1

def select_human(users):
    user1, user2 = users
    if user1['userType'] == 'Bot':
        return user2
    else:
        return user1

def get_responses(thread, user):
    return [t for t in thread if t['userId'] == user['id']]

In [4]:
human_responses = []
for dialog in data:
    bot = select_bot(dialog['users'])
    human = select_human(dialog['users'])
    context = dialog['context']
    human_responses += get_responses(dialog['thread'], human)

In [5]:
human_responses

[{'text': 'Hi', 'userId': 'Bob'},
 {'text': 'Hi', 'userId': 'Bob'},
 {'text': 'What is your name?', 'userId': 'Bob'},
 {'text': 'a Thursday or Wednesday', 'userId': 'Alice'},
 {'text': 'friday', 'userId': 'Alice'},
 {'text': '13', 'userId': 'Alice'},
 {'text': 'ok', 'userId': 'Alice'},
 {'text': 'yes', 'userId': 'Alice'},
 {'text': '?', 'userId': 'Alice'},
 {'text': 'Hi, what this text is about?', 'userId': 'Alice'},
 {'text': 'Dynamics in the early mulsim world always strike me as pretty dramatic... Christianity was so obscure for a long time.',
  'userId': 'Bob'},
 {'text': 'No, I was born a couple of centuries too late :D', 'userId': 'Bob'},
 {'text': 'Yes, he wrote the Koran.', 'userId': 'Bob'},
 {'text': 'Do you know Ali?', 'userId': 'Bob'},
 {'text': '?', 'userId': 'Bob'},
 {'text': 'Goodbye', 'userId': 'Bob'},
 {'text': 'Some sort of copyright trade deal?', 'userId': 'Bob'},
 {'text': 'I thought we were supposed to discuss the article :D',
  'userId': 'Bob'},
 {'text': 'Ok ok no

In [6]:
import gensim

/usr/local/var/pyenv/versions/3.5.0/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [7]:
model = gensim.models.Word2Vec.load_word2vec_format('/data/shared/word_vectors/GoogleNews-vectors-negative300.bin', binary=True)

In [8]:
import nltk
import numpy as np

In [9]:
human_tokens = [nltk.word_tokenize(r['text']) for r in human_responses]

In [10]:
mean_vecs = []
for sent_tokens in human_tokens:
    mean_w2v = np.zeros(300)
    for token in sent_tokens:
        if token in model:
            mean_w2v += model[token]
    mean_w2v = mean_w2v / len(sent_tokens)
    mean_vecs.append(mean_w2v)

In [11]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(mean_vecs)

In [12]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(X_std)

In [14]:
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

traces = []

trace = Scatter(
    x=Y_sklearn[:, 0],
    y=Y_sklearn[:, 1],
    mode='markers',
    name='test',
    marker=Marker(
        size=12,
        line=Line(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5),
        opacity=0.8))
traces.append(trace)


data = Data(traces)
layout = Layout(xaxis=XAxis(title='PC1', showline=False),
                yaxis=YAxis(title='PC2', showline=False))
fig = Figure(data=data, layout=layout)
py.iplot(fig)
